In [38]:
import pandas as pd
import time
import requests
import numpy as np
import geocoder
from googlemaps import Client as GoogleMaps
import googlemaps
import gmaps
from keplergl import KeplerGl
import geopandas as gpd
api_key = "***" #for_geocoder
gmaps = googlemaps.Client(key=api_key)


In [ ]:
path = 'midgard.csv'
df = (
    pd.read_csv(path, names = ['name', 'surname', 'email', 'mailto', 'contact', 'street', 'city', 'index', 'y', 'date', 'item'])

) #give the headers

In [5]:
#create new columns
df['full_address'] = df['street']+', '+df['city']
df['long'] = ""
df['lat'] = ""
df['proper_address'] = ""
df['country'] = ""

In [ ]:
#skip the step of cleaning the address because Google will return formatted_address

In [ ]:
df_unique = df.drop_duplicates(subset=['full_address']) 

results = []
for i, row in df_unique.iterrows():
    full_address = row["full_address"]
    print(full_address)
    
    if pd.isnull(full_address):  #Google API gives error when value is nan
        continue 
    
    geocode_result = gmaps.geocode(full_address)
    if not geocode_result:    #later some other problems appeared like error 400 
        continue

    geocode_info = geocode_result[0]
    loc = geocode_info.get('geometry', {}).get('location', {})  #it gives you structured data but every key is called 'long name'
    
    if not loc:
        continue
    
    address_components = geocode_info.get('address_components', [])
    country = next((component['long_name'] for component in address_components if 'country' in component['types']), None)
    results.append({
        "row_number": row["row_number"],
        "full_address": full_address,
        "lat": loc.get('lat'),
        "long": loc.get('lng'),
        "formatted_address": geocode_info.get('formatted_address'),
        "country": country
    })

In [25]:
#merged formatted_address, lat, lng, country with a df_unique with unique values
df_unique = df_unique.merge(pd.DataFrame(results),
        on="full_address")

In [135]:
#I saw no other issues, so didn't do regex
df_unique['name'] = df_unique['name'].str.lower()
df_unique['name'] = df_unique['name'].str.strip()

df_unique['surname'] = df_unique['surname'].str.strip()
df_unique['surname'] = df_unique['surname'].str.lower()


df_unique['full_name'] = df_unique['name']+' '+df_unique['surname']

In [136]:
df_unique.to_csv('midgard_unique.csv', index=False)

In [140]:
polish_df = df_unique[df_unique['country_y'] == 'Poland']
len(polish_df)

102

In [141]:
polish_df['full_name'].nunique()

75